In [2]:
sc
sqlContext = SQLContext(sc) 

# Loading and Saving Data
* Local Filesystem 
* Amazon S3
* HDFS

# Spark SQL

## 1. Creating DataFrames
* `.toDF()`
<br>all cols: string & nullable
* `.createDataFrame(rowRDD, schema)`
    ```
    schema = StructType([
        StructField("name1", IntegerType(), nullable(default:True), metadata(default: None),
        StructField("name2", TimestampType(), nullable(default:True), metadata(default: None),
        StructField("name3", LongType(), nullable(default:True), metadata(default: None),
        StructField("name4", StringType(), nullable(default:True), metadata(default: None),
        ])
    ```
    
## 2. DataFrame API Basics
```
select()
drop()
filter(),where()
withColumnRenamed(), witColumn() (Renaming and adding columns)
orderBy(), sort()
```

## 3. SQL functions with DataFrame

## 4. Registering DataFrame in the Table Catalog 
* You can reference a DataFrame by its name by registering the DataFrame as a table. 
* Spark stores the table defitition in the table catalog.
* Save as table (2)
    1. `registerTempTable(<table_name>)` -- Depreciated 
        * disappear with the Spark session.
    2. `write.saveAsTable(<table_name>)`
        * register a table permanently.
        * The table definitions will survive your application’s restarts and are persistent.
* Once DataFrame is registered as a table, you can query its data using SQL expressions.
    * Using `sqlContext.sql(sql_query)`. 
    * Also can use spark-sql.
 
## 5. Loading and Saving Data using Spark SQL
Datatypes 
* JSON
    * Spark can automatically infer a JSON schema. 
* ORC
    * Optimized Row Columnar.
    * Columnar format. 
* Parquet
    * Columnar file-based storage format optimized for relational access.
    * Designed to be independent of any specific framework and free of unnecessary dependencies.
* Relational Databases and Other DB with JDBC

1.Loading Data with SparkSQL 
* JSON: Use `spark.read.json(<file_name>)`. 
* ORC: Use `sqlContext.read.format("orc").load(<file_name>)`. 
* Parquet: Use `spark.read.parquet(<file_name>)`.
* Relational Databases and Other DB with JDBC: Use jdbc drivers.
    <br>Spark can access several popular databases using either Hadoop connectors or custom Spark connectors
    <br>Postgres and others (MySQL) using JDBC: Spark can load data from any relational databases that supports Java Database Connectivity (JDBC) including MySQL, Postgres, and other systems.
            * Download a Postgres JDBC jar from https://jdbc.postgresql.org/download.html 
            * pyspark --driver-class-path Postgres_JDBC_location
         from pyspark.sql import DataFrameReader
         df = DataFrameReader(sqlContext).jdbc(url='jdbc:<URL>', table='<table_name>', properties=<property_list>)
         where <property_list> includes user, password and driver.
            

2.Saving Data with SparkSQL 
* SaveAsTable: `.write.format(<format>).saveAsTable(<name>)`

# Loading and Saving Data

### ex1

In [26]:
rdd = sc.textFile("s3n://697-carriexu/README.md")

In [27]:
rdd.collect()

[u'# Apache Spark',
 u'',
 u'Spark is a fast and general cluster computing system for Big Data. It provides',
 u'high-level APIs in Scala, Java, Python, and R, and an optimized engine that',
 u'supports general computation graphs for data analysis. It also supports a',
 u'rich set of higher-level tools including Spark SQL for SQL and DataFrames,',
 u'MLlib for machine learning, GraphX for graph processing,',
 u'and Spark Streaming for stream processing.',
 u'',
 u'<http://spark.apache.org/>',
 u'',
 u'',
 u'## Online Documentation',
 u'',
 u'You can find the latest Spark documentation, including a programming',
 u'guide, on the [project web page](http://spark.apache.org/documentation.html)',
 u'and [project wiki](https://cwiki.apache.org/confluence/display/SPARK).',
 u'This README file only contains basic setup instructions.',
 u'',
 u'## Building Spark',
 u'',
 u'Spark is built using [Apache Maven](http://maven.apache.org/).',
 u'To build Spark and its example programs, run:',
 u'',
 

# Spark SQL

## 1. Creating DataFrames
converting existed RDD
<br>load text, parse the line, make elements into tuple
* `.toDF()`
<br>all cols: string & nullable
* `.createDataFrame(rowRDD, schema=None, samplingRatio=None, verifySchema=True)`
    <br>`schema`: a StructType or list of column names
    <br>data type:
    ```
    NullType(), StringType(), BinaryType(), BooleanType(), DateType(),TimstampType(), 
    DoubleType(), FloatType(), IntegerType()
    ```
    ```
    schema = StructType([
        StructField("name1", IntegerType(), nullable(default:True), metadata(default: None),
        StructField("name2", TimestampType(), nullable(default:True), metadata(default: None),
        StructField("name3", LongType(), nullable(default:True), metadata(default: None),
        StructField("name4", StringType(), nullable(default:True), metadata(default: None),
        ])
    ```

### ex2: `.toDF()`

In [40]:
text = sc.textFile("2018-msan697-example/Data/Italian_Stack_Exchange/italianPosts.csv").map(lambda x:x.split("~"))\
    .map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8],x[9],x[10],x[11],x[12]))\
    .toDF(["commentCount", "lastActivityDate", "ownerUserId", "body", "score", "creationDate", "viewCount", "title", "tags", "answerCount", "acceptedAnswerId", "postTypeId", "id"])

In [41]:
text.printSchema()

root
 |-- commentCount: string (nullable = true)
 |-- lastActivityDate: string (nullable = true)
 |-- ownerUserId: string (nullable = true)
 |-- body: string (nullable = true)
 |-- score: string (nullable = true)
 |-- creationDate: string (nullable = true)
 |-- viewCount: string (nullable = true)
 |-- title: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- answerCount: string (nullable = true)
 |-- acceptedAnswerId: string (nullable = true)
 |-- postTypeId: string (nullable = true)
 |-- id: string (nullable = true)



In [42]:
text.show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|           4|2013-11-11 18:21:...|         17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|     null|                    |                    |       null|            null|         2|1165|
|           5|2013-11-10 20:31:...|         12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61|Cosa sapreste dir...| &lt;word-choice&gt;|          1|            null|         1|1166|
|           2|2013-11-10 20:31:...|

### ex3: `.createDataFrame()`

In [3]:
sqlContext = SQLContext(sc) 

In [4]:
itPostsRows = sc.textFile("2018-msan697-example/Data/Italian_Stack_Exchange/italianPosts.csv")

In [5]:
from pyspark.sql import Row
from datetime import datetime
def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%Y-%m-%d %H:%M:%S.%f")
  except ValueError:
    return None

def toLongSafe(inval):
  try:
    return long(inval)
  except ValueError:
    return None
    
def stringToPost(row):
  r = row.encode('utf8').split("~")
  return Row(
    toIntSafe(r[0]),
    toTimeSafe(r[1]),
    toLongSafe(r[2]),
    r[3],
    toIntSafe(r[4]),
    toTimeSafe(r[5]),
    toIntSafe(r[6]),
    r[7],
    r[8],
    toIntSafe(r[9]),
    toLongSafe(r[10]),
    toLongSafe(r[11]),
    toLongSafe(r[12]))

In [6]:
from pyspark.sql.types import *
postSchema = StructType([
  StructField("commentCount", IntegerType(), True),
  StructField("lastActivityDate", TimestampType(), True),
  StructField("ownerUserId", LongType(), True),
  StructField("body", StringType(), True),
  StructField("score", IntegerType(), True),
  StructField("creationDate", TimestampType(), True),
  StructField("viewCount", IntegerType(), True),
  StructField("title", StringType(), True),
  StructField("tags", StringType(), True),
  StructField("answerCount", IntegerType(), True),
  StructField("acceptedAnswerId", LongType(), True),
  StructField("postTypeId", LongType(), True),
  StructField("id", LongType(), False)
  ])

In [7]:
rowRDD = itPostsRows.map(lambda x: stringToPost(x))

In [8]:
itPostsDFStruct = sqlContext.createDataFrame(rowRDD, postSchema)

In [9]:
itPostsDFStruct.printSchema()

root
 |-- commentCount: integer (nullable = true)
 |-- lastActivityDate: timestamp (nullable = true)
 |-- ownerUserId: long (nullable = true)
 |-- body: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- creationDate: timestamp (nullable = true)
 |-- viewCount: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- answerCount: integer (nullable = true)
 |-- acceptedAnswerId: long (nullable = true)
 |-- postTypeId: long (nullable = true)
 |-- id: long (nullable = false)



In [10]:
itPostsDFStruct.show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|           4|2013-11-11 18:21:...|         17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|     null|                    |                    |       null|            null|         2|1165|
|           5|2013-11-10 20:31:...|         12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61|Cosa sapreste dir...| &lt;word-choice&gt;|          1|            null|         1|1166|
|           2|2013-11-10 20:31:...|

## 2. DataFrame API Basics
```
select()
drop()
filter(),where()
withColumnRenamed(), witColumn() (Renaming and adding columns)
orderBy(), sort()
```

### ex4

`select(<column_names>` or `select(dataframe[<column_name>])`

In [10]:
itPostsDFIdBody = itPostsDFStruct.select("id", "body")
itPostsDFIdBody.show()

+----+--------------------+
|  id|                body|
+----+--------------------+
|1165|&lt;p&gt;The infi...|
|1166|&lt;p&gt;Come cre...|
|1167|&lt;p&gt;Il verbo...|
|1168|&lt;p&gt;As part ...|
|1169|&lt;p&gt;&lt;em&g...|
|1170|&lt;p&gt;There's ...|
|1171|&lt;p&gt;As other...|
|1172|&lt;p&gt;The expr...|
|1173|&lt;p&gt;When I w...|
|1174|&lt;p&gt;Wow, wha...|
|1175|&lt;p&gt;Suppose ...|
|1176|&lt;p&gt;Except w...|
|1177|&lt;p&gt;Both you...|
|1178|&lt;blockquote&gt...|
|1179|&lt;p&gt;Comparin...|
|1180|&lt;p&gt;Using th...|
|1181|&lt;p&gt;I would ...|
|1182|&lt;p&gt;Putting ...|
|1183|&lt;p&gt;Many peo...|
|1184|&lt;p&gt;Sono un'...|
+----+--------------------+
only showing top 20 rows



In [11]:
itPostsDFIdBody = itPostsDFStruct.select(itPostsDFStruct["id"], itPostsDFStruct["body"])
itPostsDFIdBody.show()

+----+--------------------+
|  id|                body|
+----+--------------------+
|1165|&lt;p&gt;The infi...|
|1166|&lt;p&gt;Come cre...|
|1167|&lt;p&gt;Il verbo...|
|1168|&lt;p&gt;As part ...|
|1169|&lt;p&gt;&lt;em&g...|
|1170|&lt;p&gt;There's ...|
|1171|&lt;p&gt;As other...|
|1172|&lt;p&gt;The expr...|
|1173|&lt;p&gt;When I w...|
|1174|&lt;p&gt;Wow, wha...|
|1175|&lt;p&gt;Suppose ...|
|1176|&lt;p&gt;Except w...|
|1177|&lt;p&gt;Both you...|
|1178|&lt;blockquote&gt...|
|1179|&lt;p&gt;Comparin...|
|1180|&lt;p&gt;Using th...|
|1181|&lt;p&gt;I would ...|
|1182|&lt;p&gt;Putting ...|
|1183|&lt;p&gt;Many peo...|
|1184|&lt;p&gt;Sono un'...|
+----+--------------------+
only showing top 20 rows



In [12]:
itPostsDFStruct.drop('id').show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+
|           4|2013-11-11 18:21:...|         17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|     null|                    |                    |       null|            null|         2|
|           5|2013-11-10 20:31:...|         12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61|Cosa sapreste dir...| &lt;word-choice&gt;|          1|            null|         1|
|           2|2013-11-10 20:31:...|         17|&lt;p&gt;Il v

In [13]:
itPostsDFStruct.drop(itPostsDFStruct['id']).show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+
|           4|2013-11-11 18:21:...|         17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|     null|                    |                    |       null|            null|         2|
|           5|2013-11-10 20:31:...|         12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61|Cosa sapreste dir...| &lt;word-choice&gt;|          1|            null|         1|
|           2|2013-11-10 20:31:...|         17|&lt;p&gt;Il v

In [14]:
itPostsDFFilteredId = itPostsDFStruct.where("id > 2000 and id < 2010")
#itPostsDFFilteredId = itPostsDFStruct.filter("id > 2000 and id < 2010")
itPostsDFFilteredId.select("id", "body").show()

+----+--------------------+
|  id|                body|
+----+--------------------+
|2001|&lt;p&gt;Sardinia...|
|2002|&lt;p&gt;I am fro...|
|2003|&lt;p&gt;La rispo...|
|2004|&lt;p&gt;In itali...|
|2005|&lt;p&gt;Mi Ã¨ st...|
|2006|&lt;p&gt;âCa.â...|
|2007|&lt;p&gt;Un chimi...|
|2008|&lt;p&gt;&quot;sp...|
|2009|&lt;p&gt;Ad occhi...|
+----+--------------------+



In [18]:
itPostsDFFilteredId = itPostsDFStruct.where(itPostsDFStruct['id'] > 2000)
#itPostsDFFilteredId = itPostsDFStruct.where(itPostsDFStruct['id'] > 2000)
itPostsDFFilteredId.select("id", "body").show()

+----+--------------------+
|  id|                body|
+----+--------------------+
|2165|&lt;p&gt;Leggendo...|
|2166|&lt;p&gt;Entrambe...|
|2167|&lt;p&gt;When I w...|
|2168|&lt;p&gt;This is ...|
|2169|&lt;p&gt;On Sicil...|
|2170|&lt;p&gt;Qual Ã¨ ...|
|2171|&lt;p&gt;&lt;a hr...|
|2172|&lt;p&gt;Can you ...|
|2173|&lt;p&gt;I would ...|
|2174|&lt;p&gt;Penso si...|
|2175|&lt;p&gt;Qualcuno...|
|2176|&lt;p&gt;I am pre...|
|2177|&lt;p&gt;I would ...|
|2178|&lt;p&gt;Secondo ...|
|2180|&lt;p&gt;Thre is ...|
|2181|&lt;p&gt;Qual Ã¨ ...|
|2182|&lt;p&gt;&lt;em&g...|
|2183|&lt;p&gt;In gener...|
|2184|&lt;p&gt;Mi chied...|
|2185|&lt;p&gt;La spieg...|
+----+--------------------+
only showing top 20 rows



In [19]:
#withColumnRenamed(existing, new)
itPostsDFFilteredId.withColumnRenamed("id","selected_id")

DataFrame[commentCount: int, lastActivityDate: timestamp, ownerUserId: bigint, body: string, score: int, creationDate: timestamp, viewCount: int, title: string, tags: string, answerCount: int, acceptedAnswerId: bigint, postTypeId: bigint, selected_id: bigint]

In [20]:
#withColumn(columnName, columnExpression)
itPostsDFWithRatio = itPostsDFFilteredId.withColumn("score_div_answer",\
                                                    itPostsDFFilteredId['score']/itPostsDFFilteredId['answerCount'])

In [21]:
itPostsDFWithRatio.where(itPostsDFWithRatio['score_div_answer'].isNotNull()).show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+------------------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|  score_div_answer|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+------------------+
|           0|2014-08-13 09:09:...|        725|&lt;p&gt;Leggendo...|    4|2014-08-11 22:58:...|       35|Quando usare &quo...|&lt;grammar&gt;&l...|          1|            null|         1|2165|               4.0|
|           7|2014-08-14 13:10:...|        804|&lt;p&gt;On Sicil...|    5|2014-08-12 09:06:...|       83|What does -usa su...|&lt;etymology&gt;...|     

In [27]:
itPostsDFWithRatio.sort("id", ascending=False).select('id').show()
#itPostsDFWithRatio.orderBy("id", ascending=0).select('id').show()

+----+
|  id|
+----+
|2347|
|2346|
|2345|
|2344|
|2343|
|2342|
|2341|
|2340|
|2339|
|2337|
|2336|
|2334|
|2333|
|2332|
|2331|
|2330|
|2328|
|2326|
|2325|
|2324|
+----+
only showing top 20 rows



In [25]:
itPostsDFWithRatio.sort("id", "ownerUserId", ascending=False).select("id", "ownerUserId").show()
#itPostsDFWithRatio.orderBy("id", "ownerUserId", ascending=False).select("id", "ownerUserId").show()

+----+-----------+
|  id|ownerUserId|
+----+-----------+
|2347|         37|
|2346|        193|
|2345|        707|
|2344|         37|
|2343|        707|
|2342|        707|
|2341|        523|
|2340|        519|
|2339|        707|
|2337|        193|
|2336|        808|
|2334|        519|
|2333|         22|
|2332|        785|
|2331|        591|
|2330|        193|
|2328|        707|
|2326|        801|
|2325|        707|
|2324|        193|
+----+-----------+
only showing top 20 rows



### ex5

In [28]:
from pyspark.sql import Row
from datetime import datetime
def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%Y-%m-%d %H:%M:%S.%f")
  except ValueError:
    return None

def toLongSafe(inval):
  try:
    return long(inval)
  except ValueError:
    return None
    
def rowstorowRDD(row):
  r = row.encode('utf8').split("~")
  return Row(
    toLongSafe(r[0]),
    toTimeSafe(r[1]),
    r[2])

In [29]:
from pyspark.sql.types import *
schema = StructType([
  StructField("Id", LongType(), True),
  StructField("commentDate", TimestampType(), True),
  StructField("comment", StringType(), True)
  ])

In [30]:
rows = sc.textFile("2018-msan697-example/Data/Italian_Stack_Exchange/italianComments.csv")
rowRDD = rows.map(lambda x: rowstorowRDD(x))

In [31]:
sqlContext = SQLContext(sc) 
DF = sqlContext.createDataFrame(rowRDD, schema)

In [32]:
DF.printSchema()

root
 |-- Id: long (nullable = true)
 |-- commentDate: timestamp (nullable = true)
 |-- comment: string (nullable = true)



In [33]:
DF.show()

+---+--------------------+--------------------+
| Id|         commentDate|             comment|
+---+--------------------+--------------------+
| 18|2013-11-05 20:39:...|It's going to be ...|
|  6|2013-11-05 20:41:...|Why not &quot;IL ...|
| 18|2013-11-05 20:43:...|    Yep, added that.|
|  6|2013-11-05 20:45:...|La squadra Milan ...|
|  6|2013-11-05 20:46:...|`ExamplesLa (squa...|
| 17|2013-11-05 20:48:...|Actually, no. As ...|
|  6|2013-11-05 20:52:...|Oh, c'mon: http:/...|
| 18|2013-11-05 20:54:...|There's no citati...|
| 12|2013-11-05 20:57:...|Se il genere dei ...|
| 18|2013-11-05 21:02:...|E' un'eccezione: ...|
| 12|2013-11-05 21:03:...|I agree with Dami...|
| 17|2013-11-05 21:14:...|Agreed, even thou...|
|  6|2013-11-05 21:15:...|@GabrielePetronel...|
| 12|2013-11-05 21:15:...|+1, but, neverthe...|
|  6|2013-11-05 21:17:...|@KyriakosKyritsis...|
| 17|2013-11-05 21:18:...|@KyriakosKyritsis...|
| 17|2013-11-05 21:21:...|*Il Cairo* Ã¨ il ...|
|  5|2013-11-05 21:42:...|Conversely, En

In [34]:
import pyspark.sql.functions as func
#instr() : Returns the position of the first occurrence of substr in str
DF.filter(func.to_date(DF['commentDate'])=='2013-11-07').filter((func.instr(DF['comment'],"@Daniele") > 0)).show()

+---+--------------------+--------------------+
| Id|         commentDate|             comment|
+---+--------------------+--------------------+
| 37|2013-11-07 15:30:...|@Daniele B: I kno...|
+---+--------------------+--------------------+



### ex6

In [35]:
from pyspark.sql import Row
from datetime import datetime
def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return None

def toTimeSafe(inval):
  try:
    return datetime.strptime(inval, "%Y-%m-%d %H:%M:%S.%f")
  except ValueError:
    return None

def toLongSafe(inval):
  try:
    return long(inval)
  except ValueError:
    return None
    
def rowstorowRDD(row):
  r = row.encode('utf8').split("~")
  return Row(
      toLongSafe(r[0]),
      toLongSafe(r[1]),
      toIntSafe(r[2]),
      toTimeSafe(r[3]))

In [36]:
from pyspark.sql.types import *
schema = StructType([
    StructField("id", LongType(), False),
    StructField("postId", LongType(), False),
    StructField("voteTypeId", IntegerType(), False),
    StructField("creationTime", TimestampType(), False)
    ])

In [37]:
rows = sc.textFile("2018-msan697-example/Data/Italian_Stack_Exchange/italianVotes.csv")
rowRDD = rows.map(lambda x: rowstorowRDD(x))

In [38]:
sqlContext = SQLContext(sc) 
DF = sqlContext.createDataFrame(rowRDD, schema)

In [39]:
DF.printSchema()

root
 |-- id: long (nullable = false)
 |-- postId: long (nullable = false)
 |-- voteTypeId: integer (nullable = false)
 |-- creationTime: timestamp (nullable = false)



In [40]:
DF.show()

+----+------+----------+-------------------+
|  id|postId|voteTypeId|       creationTime|
+----+------+----------+-------------------+
|2657|   135|         2|2013-11-22 00:00:00|
|2658|   142|         2|2013-11-22 00:00:00|
|2659|   142|         1|2013-11-22 00:00:00|
|2660|   140|         2|2013-11-22 00:00:00|
|2661|   140|         1|2013-11-22 00:00:00|
|2662|  1354|         2|2013-11-22 00:00:00|
|2663|  1356|         2|2013-11-22 00:00:00|
|2664|  1353|         2|2013-11-22 00:00:00|
|2665|  1351|         2|2013-11-22 00:00:00|
|2667|  1357|         2|2013-11-22 00:00:00|
|2668|  1357|         2|2013-11-22 00:00:00|
|2669|  1351|         2|2013-11-22 00:00:00|
|2670|  1351|         1|2013-11-22 00:00:00|
|2671|  1352|         2|2013-11-22 00:00:00|
|2672|  1349|         2|2013-11-22 00:00:00|
|2673|  1357|         2|2013-11-22 00:00:00|
|2674|  1342|         2|2013-11-22 00:00:00|
|2675|  1340|         2|2013-11-22 00:00:00|
|2676|  1321|         2|2013-11-22 00:00:00|
|2677|  13

In [45]:
DF.where("voteTypeId == 1").sort('creationTime', ascending=True).show()
#DF.where(DF['voteTypeId'] == 1).sort('creationTime', ascending=True).show()

+----+------+----------+-------------------+
|  id|postId|voteTypeId|       creationTime|
+----+------+----------+-------------------+
|  53|    12|         1|2013-11-05 00:00:00|
|  46|     2|         1|2013-11-05 00:00:00|
| 195|    36|         1|2013-11-06 00:00:00|
| 232|    55|         1|2013-11-06 00:00:00|
| 220|    30|         1|2013-11-06 00:00:00|
| 216|    52|         1|2013-11-06 00:00:00|
| 181|    45|         1|2013-11-06 00:00:00|
| 364|    77|         1|2013-11-07 00:00:00|
| 380|    84|         1|2013-11-07 00:00:00|
| 375|    83|         1|2013-11-07 00:00:00|
| 501|    88|         1|2013-11-08 00:00:00|
|1651|  1158|         1|2013-11-10 00:00:00|
|1672|   106|         1|2013-11-10 00:00:00|
|1781|  1170|         1|2013-11-11 00:00:00|
|1895|  1212|         1|2013-11-12 00:00:00|
|1932|  1235|         1|2013-11-12 00:00:00|
|1979|  1228|         1|2013-11-13 00:00:00|
|2011|  1234|         1|2013-11-13 00:00:00|
|1982|  1216|         1|2013-11-13 00:00:00|
|2053|    

## 3. SQL functions with DataFrame
* Scalar functions : Return a single value for each row based on calculations on one or more columns.
    * Math – abs, log, etc.
    * String – length, concat, trim, etc.
    * Time – year, date_add, datediff, next_day
* Aggregate functions : Return a single value for a group of rows.
* Window functions : Return several values for a group of rows. Ex:lags
* User-defined functions (UDF) : Generate custom scalar or aggregate functions.

### ex7

1.Add a column called “duration”, which is difference between ‘lastActivityDate’ and ‘creationDate’ and sort the DataFrame by ‘duration’ in descending order.

In [50]:
from pyspark.sql.functions import *
#Scalar functions
itPostsDFStruct.withColumn('duration',datediff('lastActivityDate','creationDate')).sort('duration', ascending=False).show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+--------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|duration|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+--------+
|           3|2014-09-11 14:37:...|         63|&lt;p&gt;The plur...|    5|2013-11-12 13:34:...|       59|Why is the plural...|&lt;plural&gt;&lt...|          1|            1227|         1|1221|     303|
|           4|2014-09-09 08:54:...|         63|&lt;p&gt;Some wor...|    4|2013-11-12 11:03:...|       80|When is the lette...|&lt;nouns&gt;&lt;...|          2|            1207|         1|1205|

2.Calculate an average score per owner and sort data by average scores in descending order.

In [53]:
#Aggregate functions
itPostsDFStruct.groupBy('ownerUserId').avg('score').sort('avg(score)', ascending=False).show()

+-----------+-----------------+
|ownerUserId|       avg(score)|
+-----------+-----------------+
|          6|             15.0|
|        570|             15.0|
|        730|             12.0|
|        154|             11.0|
|        729|             11.0|
|        220|             10.0|
|        217|             10.0|
|        656|              9.0|
|        445|              9.0|
|        116|              9.0|
|         48|8.666666666666666|
|        114|              8.5|
|        124|8.333333333333334|
|         57|              8.3|
|        590|              8.0|
|        504|              8.0|
|        158|              8.0|
|         85|              8.0|
|         17|7.967741935483871|
|         56|              7.8|
+-----------+-----------------+
only showing top 20 rows



### Window functions
1. Construct a column definition with an aggregate function or window functions.
2. Build a window specification and use it as an argument to over() function.
    1. Define the partition using partitionBy() function.
    2. Specify ordering in the partition using orderBy().

### ex8
find the id of its owner’s previous question by creation date.

In [61]:
from pyspark.sql.window import Window

postsDf = itPostsDFStruct
winDf = postsDf.filter(postsDf.postTypeId == 1)\
    .select(postsDf.ownerUserId, postsDf.id, postsDf.creationDate, 
            lag(postsDf.id, 1).over(Window.partitionBy(postsDf.ownerUserId).orderBy(postsDf.creationDate)).alias("prev"))\
    .orderBy(postsDf.ownerUserId, postsDf.id).show()

+-----------+----+--------------------+----+
|ownerUserId|  id|        creationDate|prev|
+-----------+----+--------------------+----+
|          4|1637|2014-01-24 06:51:...|null|
|          8|   1|2013-11-05 20:22:...|null|
|          8| 112|2013-11-08 13:14:...|   1|
|          8|1192|2013-11-11 21:01:...| 112|
|          8|1276|2013-11-15 16:09:...|1192|
|          8|1321|2013-11-20 16:42:...|1276|
|          8|1365|2013-11-23 09:09:...|1321|
|         12|  11|2013-11-05 21:30:...|null|
|         12|  17|2013-11-05 22:17:...|  11|
|         12|  18|2013-11-05 22:34:...|  17|
|         12|  19|2013-11-05 22:38:...|  18|
|         12|  63|2013-11-06 17:54:...|  19|
|         12|  65|2013-11-06 18:07:...|  63|
|         12|  69|2013-11-06 19:41:...|  65|
|         12|  70|2013-11-06 20:35:...|  69|
|         12|  89|2013-11-07 19:22:...|  70|
|         12|  94|2013-11-07 20:42:...|  89|
|         12| 107|2013-11-08 08:27:...|  94|
|         12| 122|2013-11-08 20:55:...| 107|
|         

find the id of its owner’s previous and next question by creation date.

In [62]:
winDf = postsDf.filter(postsDf.postTypeId == 1)\
    .select(postsDf.ownerUserId, postsDf.id, postsDf.creationDate, 
        lag(postsDf.id, 1).over(Window.partitionBy(postsDf.ownerUserId).orderBy(postsDf.creationDate)).alias("prev"), 
        lead(postsDf.id, 1).over(Window.partitionBy(postsDf.ownerUserId).orderBy(postsDf.creationDate)).alias("next"))\
    .orderBy(postsDf.ownerUserId, postsDf.id).show()

+-----------+----+--------------------+----+----+
|ownerUserId|  id|        creationDate|prev|next|
+-----------+----+--------------------+----+----+
|          4|1637|2014-01-24 06:51:...|null|null|
|          8|   1|2013-11-05 20:22:...|null| 112|
|          8| 112|2013-11-08 13:14:...|   1|1192|
|          8|1192|2013-11-11 21:01:...| 112|1276|
|          8|1276|2013-11-15 16:09:...|1192|1321|
|          8|1321|2013-11-20 16:42:...|1276|1365|
|          8|1365|2013-11-23 09:09:...|1321|null|
|         12|  11|2013-11-05 21:30:...|null|  17|
|         12|  17|2013-11-05 22:17:...|  11|  18|
|         12|  18|2013-11-05 22:34:...|  17|  19|
|         12|  19|2013-11-05 22:38:...|  18|  63|
|         12|  63|2013-11-06 17:54:...|  19|  65|
|         12|  65|2013-11-06 18:07:...|  63|  69|
|         12|  69|2013-11-06 19:41:...|  65|  70|
|         12|  70|2013-11-06 20:35:...|  69|  89|
|         12|  89|2013-11-07 19:22:...|  70|  94|
|         12|  94|2013-11-07 20:42:...|  89| 107|


### User-defined functions (UDF)

### ex9
Create a UDF called countTags which counts the number of &lt; and count the number of them in the tags column.

In [63]:
from pyspark.sql.functions import *

countTags = udf(lambda (x) : x.count("&lt;"))
itPostsDFStruct.select('id', countTags('tags')).show()

+----+--------------+
|  id|<lambda>(tags)|
+----+--------------+
|1165|             0|
|1166|             1|
|1167|             0|
|1168|             3|
|1169|             0|
|1170|             0|
|1171|             0|
|1172|             0|
|1173|             2|
|1174|             0|
|1175|             3|
|1176|             2|
|1177|             0|
|1178|             0|
|1179|             2|
|1180|             0|
|1181|             0|
|1182|             0|
|1183|             2|
|1184|             2|
+----+--------------+
only showing top 20 rows



### ex10
Create a UDF called scoreString which discretize scores 
    * Lower than 10 : low
    * Between 10 and 20 : med
    * Higher than 20 : high
and show the score strings.

In [64]:
from pyspark.sql.functions import *

def s(x):
    if x < 10:
        return 'low'
    if x >= 10 and x <= 20:
        return 'med'
    if x > 20:
        return 'high'
    
scoreString = udf(lambda (x): s(x))
itPostsDFStruct.select('id','score', scoreString('score')).show()

+----+-----+---------------+
|  id|score|<lambda>(score)|
+----+-----+---------------+
|1165|   23|           high|
|1166|    1|            low|
|1167|    5|            low|
|1168|   11|            med|
|1169|    3|            low|
|1170|    8|            low|
|1171|    3|            low|
|1172|    1|            low|
|1173|    5|            low|
|1174|    5|            low|
|1175|    4|            low|
|1176|    3|            low|
|1177|    6|            low|
|1178|    1|            low|
|1179|    3|            low|
|1180|    5|            low|
|1181|    8|            low|
|1182|   11|            med|
|1183|    6|            low|
|1184|    7|            low|
+----+-----+---------------+
only showing top 20 rows



**groupBy()**
* take column names or a list of column objects.
* Return GroupedData Object.
* Can use an aggregate function or agg(list_of_aggregate_fuctions).

### ex11
Calculate min, max, average score per ownerUserId.

In [65]:
itPostsDFStruct.select('ownerUserId','score')\
    .groupBy('ownerUserId').agg(max('score'),min('score'),avg('score')).show()

+-----------+----------+----------+------------------+
|ownerUserId|max(score)|min(score)|        avg(score)|
+-----------+----------+----------+------------------+
|        270|         1|         1|               1.0|
|        730|        12|        12|              12.0|
|        720|         1|         1|               1.0|
|         19|        10|        -1| 3.076923076923077|
|        348|         5|         5|               5.0|
|        415|         5|         1|2.1666666666666665|
|        656|         9|         9|               9.0|
|        736|         1|         1|               1.0|
|         22|        19|         0| 4.886363636363637|
|        198|         5|         5|               5.0|
|         77|         9|         0| 5.166666666666667|
|        677|         6|         6|               6.0|
|        202|         3|        -1|               1.0|
|        228|         2|         2|               2.0|
|        624|         4|         4|               4.0|
|        5

In [66]:
itPostsDFStruct.select('ownerUserId','score')\
    .groupBy('ownerUserId').max('score').show()

+-----------+----------+
|ownerUserId|max(score)|
+-----------+----------+
|        270|         1|
|        730|        12|
|        720|         1|
|         19|        10|
|        348|         5|
|        415|         5|
|        656|         9|
|        736|         1|
|         22|        19|
|        198|         5|
|         77|         9|
|        677|         6|
|        202|         3|
|        228|         2|
|        624|         4|
|        570|        15|
|        421|         9|
|        229|         8|
|        190|         4|
|         57|        13|
+-----------+----------+
only showing top 20 rows



**Grouping and Joining DataFrame**
<br>`join(dataframe, condition, join_type)`
    * Join types : inner (default), outer, left_outer, right_outer, leftsemi
  

### ex13

In [67]:
itPostsDFStruct.join(DF, itPostsDFStruct.id == DF.postId, 'inner').collect()

[Row(commentCount=1, lastActivityDate=datetime.datetime(2013, 11, 6, 0, 22, 44, 67000), ownerUserId=18, body=u'&lt;p&gt;&quot;Degli&quot; in this context means &quot;a certain quantity of, some&quot;, so the two examples have different meanings:&lt;/p&gt;&lt;blockquote&gt;  &lt;ol&gt;  &lt;li&gt;I went to the restaurant with friends&lt;/li&gt;  &lt;li&gt;I went to the restaurant with some friends&lt;/li&gt;  &lt;/ol&gt;&lt;/blockquote&gt;&lt;p&gt;The change in meaning is analogous as you would have in English.&lt;/p&gt;', score=3, creationDate=datetime.datetime(2013, 11, 6, 0, 5, 52, 740000), viewCount=None, title=u'', tags=u'', answerCount=None, acceptedAnswerId=None, postTypeId=2, id=26, id=62, postId=26, voteTypeId=2, creationTime=datetime.datetime(2013, 11, 6, 0, 0)),
 Row(commentCount=1, lastActivityDate=datetime.datetime(2013, 11, 6, 0, 22, 44, 67000), ownerUserId=18, body=u'&lt;p&gt;&quot;Degli&quot; in this context means &quot;a certain quantity of, some&quot;, so the two examp

## 4. Registering DataFrame in the Table Catalog 
* You can reference a DataFrame by its name by registering the DataFrame as a table. 
* Spark stores the table defitition in the table catalog.
* Save as table (2)
    1. `registerTempTable(<table_name>)` -- Depreciated 
        * disappear with the Spark session.
    2. `write.saveAsTable(<table_name>)`
        * register a table permanently.
        * The table definitions will survive your application’s restarts and are persistent.
* Once DataFrame is registered as a table, you can query its data using SQL expressions.
    * Using `sqlContext.sql(sql_query)`. 
    * Also can use spark-sql.

### ex14_15
Save Post DataFrame as “Posts”

In [11]:
itPostsDFStruct.write.saveAsTable("Posts")

Run “select * from posts” query.

In [12]:
resultDf = sqlContext.sql("select * from Posts").show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|           4|2013-11-11 18:21:...|         17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|     null|                    |                    |       null|            null|         2|1165|
|           5|2013-11-10 20:31:...|         12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61|Cosa sapreste dir...| &lt;word-choice&gt;|          1|            null|         1|1166|
|           2|2013-11-10 20:31:...|

## 5. Loading and Saving Data using Spark SQL
Datatypes 
* JSON
    * Spark can automatically infer a JSON schema. 
* ORC
    * Optimized Row Columnar.
    * Columnar format. 
* Parquet
    * Columnar file-based storage format optimized for relational access.
    * Designed to be independent of any specific framework and free of unnecessary dependencies.
* Relational Databases and Other DB with JDBC

1.Loading Data with SparkSQL 
* JSON: Use `spark.read.json(<file_name>)`. 
* ORC: Use `sqlContext.read.format("orc").load(<file_name>)`. 
* Parquet: Use `spark.read.parquet(<file_name>)`.
* Relational Databases and Other DB with JDBC: Use jdbc drivers.
    <br>Spark can access several popular databases using either Hadoop connectors or custom Spark connectors
    <br>Postgres and others (MySQL) using JDBC: Spark can load data from any relational databases that supports Java Database Connectivity (JDBC) including MySQL, Postgres, and other systems.
            * Download a Postgres JDBC jar from https://jdbc.postgresql.org/download.html 
            * pyspark --driver-class-path Postgres_JDBC_location
         from pyspark.sql import DataFrameReader
         df = DataFrameReader(sqlContext).jdbc(url='jdbc:<URL>', table='<table_name>', properties=<property_list>)
         where <property_list> includes user, password and driver.
            

2.Saving Data with SparkSQL 
* SaveAsTable: `.write.format(<format>).saveAsTable(<name>)`

### ex16
Read s3n://usfca-msan694/world_bank_project.json as a DataFrame.

In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

world_bank_prj = spark.read.json("s3n://usfca-msan694/world_bank_project.json")

### ex17
Save posts in the table catalog to a json_out folder in your S3 bucket.

In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

postDf = sqlContext.sql("select * from Posts")
postDf.write.format("json").saveAsTable("post_json",path="s3n://usfca-msan694/json_out")